In [2]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import hopsworks
import pandas as pd
import numpy as np

/Users/berkan/Desktop/wine_quality/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
#read the wine quality data set
wine_df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv")
wine_df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
wine_df['type'].value_counts()

type
white    4898
red      1599
Name: count, dtype: int64

In [4]:
numerical_col = wine_df.select_dtypes(np.number).columns  #list(wine_df.columns[1:11])
categorical_col = wine_df.select_dtypes(object).columns #["type", "quality"]
wine_df[numerical_col] = wine_df[numerical_col].fillna(wine_df[numerical_col].mean())
wine_df["type"] = wine_df["type"].fillna("unknown")

In [5]:
#-Transform categorical variables into numerical variables
wine_df['type'].replace(['white', 'red', 'unknown'], [0, 1, 2], inplace=True)
wine_df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.450000,8.8,6
1,0,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.490000,9.5,6
2,0,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.440000,10.1,6
3,0,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.400000,9.9,6
4,0,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.400000,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,1,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.580000,10.5,5
6493,1,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.531215,11.2,6
6494,1,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.750000,11.0,6
6495,1,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.710000,10.2,5


In [6]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
# Use mutual information for feature selection 
selector = SelectKBest(mutual_info_classif, k=10)
X = wine_df.drop('quality', axis=1)
y = wine_df['quality']
X_selected = selector.fit_transform(X, y)

# Get selected feature indices
selected_indices = selector.get_support()

# Get the selected feature names
selected_features = X.columns[selected_indices]
dropped_features = X.columns[~selected_indices]

# Print the selected features
print("Selected features:", selected_features)
print("Dropped features:", dropped_features)

Selected features: Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'sulphates', 'alcohol'],
      dtype='object')
Dropped features: Index(['type', 'pH'], dtype='object')


In [7]:
wine_df = wine_df.drop(dropped_features, axis=1)
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,0.450000,8.8,6
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,0.490000,9.5,6
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,0.440000,10.1,6
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,0.580000,10.5,5
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,0.531215,11.2,6
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,0.750000,11.0,6
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,0.710000,10.2,5


In [8]:
wine_df.columns = wine_df.columns.str.replace(' ', '_')
wine_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,0.450000,8.8,6
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,0.490000,9.5,6
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,0.440000,10.1,6
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,0.400000,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,0.580000,10.5,5
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,0.531215,11.2,6
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,0.750000,11.0,6
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,0.710000,10.2,5


Write the features to the feature store as a Feature Group

wine_fg = fs.get_or_create_feature_group(
    name="wine_quality",
    version=1,
    primary_key=list(wine_df.columns), 
    description="Wine quality dataset")
wine_fg.insert(wine_df)

In [140]:
#read the wine quality data set
df = pd.read_csv("https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv")
df.drop(['type','pH'],inplace= True,axis = 1)
df.columns = df.columns.str.replace(' ',"_")

In [141]:
result = df.groupby('quality').agg(['mean', 'std']).T
result

quality                             3           4           5           6  \
fixed_acidity        mean    7.853333    7.288889    7.329348    7.178037   
                     std     1.747853    1.264825    1.259910    1.291691   
volatile_acidity     mean    0.517000    0.457963    0.389768    0.313731   
                     std     0.341913    0.229311    0.178591    0.147025   
citric_acid          mean    0.281000    0.272315    0.307721    0.323786   
                     std     0.173510    0.182079    0.160466    0.142341   
residual_sugar       mean    5.140000    4.153704    5.804116    5.551182   
                     std     4.730721    3.812136    4.994311    4.888507   
chlorides            mean    0.077033    0.060158    0.064666    0.054168   
                     std     0.062014    0.047153    0.042007    0.030844   
free_sulfur_dioxide  mean   39.216667   20.636574   30.237371   31.165021   
                     std    59.990327   18.865524   18.565479   16.835507   
total_sulfur_dioxide mean  122.033333  103.432870  120.839102  115.410790   
                     std   112.141762   61.282714   60.782859   55.454390   
density              mean    0.995744    0.994833    0.995849    0.994558   
                     std     0.002833    0.002475    0.002439    0.003035   
sulphates            mean    0.506333    0.504884    0.526416    0.532466   
                     std     0.126994    0.164433    0.141680    0.146730   
alcohol              mean   10.215000   10.180093    9.837783   10.587553   
                     std     1.106497    0.985923    0.814416    1.126491   

quality                             7           8           9  
fixed_acidity        mean    7.128962    6.838542    7.420000  
                     std     1.372141    1.149311    0.983362  
volatile_acidity     mean    0.288895    0.291010    0.298000  
                     std     0.116784    0.119357    0.057619  
citric_acid          mean    0.334764    0.332539    0.386000  
                     std     0.111471    0.102446    0.082037  
residual_sugar       mean    4.733952    5.382902    4.120000  
                     std     4.041440    4.175467    3.759255  
chlorides            mean    0.045272    0.041124    0.027400  
                     std     0.021786    0.015694    0.007436  
free_sulfur_dioxide  mean   30.422150   34.533679   33.400000  
                     std    14.925261   17.195268   13.427584  
total_sulfur_dioxide mean  108.498610  117.518135  116.000000  
                     std    47.943716   42.135241   19.824228  
density              mean    0.993126    0.992514    0.991460  
                     std     0.003020    0.002880    0.003118  
sulphates            mean    0.547025    0.512487    0.466000  
                     std     0.160436    0.165891    0.092628  
alcohol              mean   11.386006   11.678756   12.180000  
                     std     1.199337    1.278921    1.013410

In [112]:
random_quality = np.random.randint(3,10)

4

In [143]:
distinct_features  = set()
for i in result.index:
    distinct_features.add(i[0])


In [147]:
made_up_data = {}
for feature in distinct_features:
    mean,std = result.iloc[:,0][feature]
    value = np.random.normal(mean,std,1)
    if value < 0:
        value = np.abs(value)
    made_up_data[feature] = value

made_up_data

{'free_sulfur_dioxide': array([97.3812858]),
 'residual_sugar': array([7.53206759]),
 'total_sulfur_dioxide': array([263.11696415]),
 'alcohol': array([10.04846945]),
 'fixed_acidity': array([9.31437989]),
 'density': array([0.99784368]),
 'citric_acid': array([0.22246415]),
 'volatile_acidity': array([0.64644329]),
 'sulphates': array([0.27164506]),
 'chlorides': array([0.12933479])}

In [148]:
df =pd.DataFrame(made_up_data)
df['quality'] = random_quality

In [149]:
df

,free_sulfur_dioxide,residual_sugar,total_sulfur_dioxide,alcohol,fixed_acidity,density,citric_acid,volatile_acidity,sulphates,chlorides,quality
0,97.381286,7.532068,263.116964,10.048469,9.31438,0.997844,0.222464,0.646443,0.271645,0.129335,4
